In [2]:
import brax
import jax
from brax import envs
from diversity_algorithms.controllers.fixed_structure_nn_flax import SimpleNeuralControllerFlax

In [3]:
env = envs.create("ant")
env = envs.wrapper.wrap_for_training(env)
key = jax.random.PRNGKey(0)
pop_size = 100
controller = SimpleNeuralControllerFlax(env.observation_size, env.action_size, n_hidden_layers=2, n_neurons_per_hidden=10)

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [4]:
init_keys = jax.random.split(key, 100)
init_state = env.reset(init_keys)

In [5]:
shape = controller.get_parameters().shape

In [6]:
array_params = jax.vmap(lambda key: jax.random.uniform(key, shape))(init_keys)
params = jax.vmap(controller.array_to_fdict)(array_params)

In [7]:
def eval(init_states, params):
	def eval_step(carry, _):
		states = carry
		actions = jax.vmap(jax.jit(controller.predict))(params, states.obs)
		next_states = env.step(states, actions)
		return (next_states), (next_states.reward, next_states.metrics)
	return jax.lax.scan(eval_step, init_states, None, length=1000)

In [8]:
final_states, (rewards, metrics) = eval(init_state, params)